In [70]:
%load_ext autoreload
%autoreload 2

import numpy as np
from domain import Domain
from operators import calc_div, central_diff2, diff_sbp21, diff_sbp42
from vecmath import calc_c_norm, calc_l2_norm, get_sbp42_weights
from vizualization import surf_field

%matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Численное исследование сходимости

Протестируем сходимость оператора дивергенции. В качестве тестовых функций будем использовать поля
\begin{align}
u = 2\cos{2x}\cos3y, \quad v=-3\sin{2x}\sin{3y}.
\end{align}
Точное значение дивергенции такого поля скорости:
\begin{align}
-13 \sin{2x}\cos{3y}.
\end{align}

In [76]:
# списки для хранения норм ошибок
c_err_norms = []
l2_err_norms = []

# будем считать нормы ошибки при различном количестве узлов сетки
n_points = (64, 128, 256, 512)
for n in (n_points):
    domain = Domain(xs = 0.0, xe = 2 * np.pi, nx = n, \
                    ys = 0.0, ye = 2 * np.pi, ny = n)

    u =  2 * np.cos(2*domain.xx) * np.cos(3*domain.yy)
    v = -3 * np.sin(2*domain.xx) * np.sin(3*domain.yy)
    div_exact = -13 * np.sin(2*domain.xx) * np.cos(3*domain.yy)

    div = calc_div(u, v, domain, diff_method=diff_sbp42)
    
    err = div - div_exact
    
    c_err_norms.append(calc_c_norm(err, domain) / calc_c_norm(div_exact, domain))
    l2_err_norms.append(calc_l2_norm(err, domain, get_sbp42_weights) / calc_l2_norm(div_exact, domain, get_sbp42_weights))

c_norm_convergence_rate = -np.polyfit(np.log(n_points), np.log(c_err_norms), deg=1)[0]
l2_norm_convergence_rate = -np.polyfit(np.log(n_points), np.log(l2_err_norms), deg=1)[0]

print(f"Convergence rate in C norm = {c_norm_convergence_rate}")
print(f"Convergence rate in l2 norm = {l2_norm_convergence_rate}")

Convergence rate in C norm = 1.9784180071022488
Convergence rate in l2 norm = 2.477778273810169
